In [9]:
import pandas as pd
from assign_unique_ids_functions import keep_duplicates, remove_duplicates
pd.options.display.max_columns = 999

In [10]:
um = pd.read_csv('../input/ase-units.csv.gz')
uhc = ['UID', 'Unit', 'Date', 'Event']

In [11]:
um.shape

(114630, 5)

In [12]:
um[um['Unit'].isnull()].shape

(841, 5)

In [13]:
um.dropna(thresh=3, axis=0, inplace=True)
um.shape

(113789, 5)

In [14]:
um.head()

ase-units_ID  Unit Effective.Date    End.Date  UID
0             1  14.0     2007-03-29  2007-03-28    1
1             1  14.0     2007-03-29         NaN    1
2             2  15.0     2007-03-29  2007-03-28    2
3             2  15.0     2007-03-29         NaN    2
4             3   1.0     1976-08-19  1976-08-20    3

In [15]:
ums = um[['UID', 'Unit', 'Effective.Date']]
ums.insert(3, 'Event', 1)
ums.columns = uhc
ums.head()

UID  Unit        Date  Event
0    1  14.0  2007-03-29      1
1    1  14.0  2007-03-29      1
2    2  15.0  2007-03-29      1
3    2  15.0  2007-03-29      1
4    3   1.0  1976-08-19      1

In [16]:
ume = um[['UID', 'Unit', 'End.Date']]
ume.insert(3, 'Event', -1)
ume.columns = uhc
ume.head()

UID  Unit        Date  Event
0    1  14.0  2007-03-28     -1
1    1  14.0         NaN     -1
2    2  15.0  2007-03-28     -1
3    2  15.0         NaN     -1
4    3   1.0  1976-08-20     -1

In [20]:
uh = pd.concat([ums, ume])

In [21]:
uh.drop_duplicates(inplace=True)

In [ ]:
max(uh['Date'].dropna())

In [22]:
uh.head()

UID  Unit        Date  Event
0    1  14.0  2007-03-29      1
2    2  15.0  2007-03-29      1
4    3   1.0  1976-08-19      1
5    3   4.0  1971-01-16      1
6    3  22.0  1970-06-15      1

In [23]:
uh.sort_values(['UID', 'Date', 'Event'], ascending=False, na_position='first', inplace=True)

In [24]:
uh.reset_index(drop=True,inplace=True)

In [25]:
uh.tail(10)

UID  Unit        Date  Event
221387    3   4.0  1976-08-18     -1
221388    3   4.0  1971-01-16      1
221389    3  22.0  1971-01-15     -1
221390    3  22.0  1970-06-15      1
221391    2  15.0         NaN     -1
221392    2  15.0  2007-03-29      1
221393    2  15.0  2007-03-28     -1
221394    1  14.0         NaN     -1
221395    1  14.0  2007-03-29      1
221396    1  14.0  2007-03-28     -1

In [26]:
uh.head(10)

UID   Unit        Date  Event
0  32016  602.0         NaN     -1
1  32016    5.0  1979-10-14     -1
2  32016  602.0  1971-02-24      1
3  32016    5.0  1971-02-22      1
4  32015   50.0         NaN     -1
5  32015   50.0  2010-11-11      1
6  32015  123.0  2010-11-10     -1
7  32015  123.0  2007-03-01      1
8  32015  124.0  2007-02-28     -1
9  32015  276.0  2007-02-28     -1

In [28]:
def number_groups(df, group_id):
    df_nums = []
    current_id = 0
    num = 1
    for i in df[group_id]:
        if i != current_id:
            num = 1
            current_id = i
        else:
            num +=1
        df_nums.append(num)
    df.insert(0, 'Number', df_nums)
    return df

In [29]:
uh = number_groups(uh, 'UID')

In [30]:
uh.sort_values(['UID', 'Date'], na_position='last',ascending=True, inplace=True)

In [33]:
uh.reset_index(inplace=True)

In [34]:
tt = uh[['UID','Event']].groupby('UID', as_index=False).apply(lambda x: x['Event'].cumsum()).reset_index().iloc[:, 2]

In [35]:
uh['Event.CS'] = tt

In [36]:
uh.drop('index', axis=1, inplace=True)

In [39]:
problem_uids = uh[abs(uh['Event.CS']) > 1]['UID'].drop_duplicates()

In [40]:
len(problem_uids)

278

In [38]:
uh[uh['UID'].isin(problem_uids)]

Number    UID   Unit        Date  Event  Event.CS
666         11     85   44.0  1955-02-01      1         1
667         12     85  148.0  1955-02-01      1         2
668         10     85   44.0  1956-07-31     -1         1
669          9     85  152.0  1965-01-16      1         2
670          8     85  152.0  1966-01-05     -1         1
671          7     85   11.0  1966-01-06      1         2
672          6     85   11.0  1966-03-02     -1         1
673          5     85   86.0  1966-03-03      1         2
674          4     85   86.0  1967-01-04     -1         1
675          3     85   86.0  1967-04-27      1         2
676          2     85   86.0  1970-02-04     -1         1
677          1     85  148.0         NaN     -1         0
740         16     90   44.0  1987-03-09      1         1
741         15     90   44.0  1988-03-02     -1         0
742         14     90    9.0  1988-03-03      1         1
743         13     90    9.0  1991-08-14     -1         0
744         12     90   92.0  1991-08-15      1         1
745         11     90   92.0  2000-12-06     -1         0
746         10     90  188.0  2000-12-07      1         1
747          9     90  188.0  2002-12-31     -1         0
748          7     90  188.0  2003-01-01      1         1
749          8     90  196.0  2003-01-01      1         2
750          5     90  188.0  2005-02-02     -1         1
751          6     90  196.0  2005-02-02     -1         0
752          4     90   22.0  2005-02-03      1         1
753          3     90   22.0  2011-07-20     -1         0
754          2     90  140.0  2011-07-21      1         1
755          1     90  140.0         NaN     -1         0
1866        12    216   44.0  1955-08-22      1         1
1867        11    216   44.0  1956-02-21     -1         0
...        ...    ...    ...         ...    ...       ...
220775      26  31843   25.0  1958-06-16      1         1
220776      25  31843   25.0  1959-04-15     -1         0
220777      24  31843  161.0  1959-04-16      1         1
220778      23  31843  161.0  1960-05-15     -1         0
220779      22  31843  134.0  1960-05-16      1         1
220780      21  31843  134.0  1966-06-30     -1         0
220781      20  31843  135.0  1966-07-01      1         1
220782      19  31843  135.0  1968-10-09     -1         0
220783      18  31843    7.0  1968-10-10      1         1
220784      17  31843    7.0  1969-03-05     -1         0
220785      16  31843  541.0  1969-03-06      1         1
220786      15  31843  541.0  1973-01-31     -1         0
220787      13  31843    7.0  1973-02-01      1         1
220788      14  31843  543.0  1973-02-01      1         2
220789      11  31843    7.0  1974-07-17     -1         1
220790      12  31843  543.0  1974-07-17     -1         0
220791      10  31843  175.0  1979-10-11      1         1
220792       9  31843  175.0  1980-09-17     -1         0
220793       8  31843  189.0  1981-01-01      1         1
220794       7  31843  189.0  1981-01-07     -1         0
220795       6  31843    6.0  1981-01-08      1         1
220796       5  31843    6.0  1981-02-04     -1         0
220797       4  31843    8.0  1981-02-05      1         1
220798       3  31843    8.0  1981-02-06     -1         0
220799       2  31843  165.0  1981-02-07      1         1
220800       1  31843  165.0         NaN     -1         0
221393       4  32016    5.0  1971-02-22      1         1
221394       3  32016  602.0  1971-02-24      1         2
221395       2  32016    5.0  1979-10-14     -1         1
221396       1  32016  602.0         NaN     -1         0

[4138 rows x 6 columns]

In [41]:
um[um['UID'].isin(problem_uids)][['UID','ase-units_ID']].to_csv('problem_ase.csv', index=False)